In [29]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import statements required for Plotly 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, log_loss, classification_report)
from imblearn.over_sampling import SMOTE
import xgboost

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [66]:
attrition = pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
attrition.head()
attrition.drop('EmployeeNumber',axis=1,inplace=True)
test.drop('EmployeeNumber',axis=1,inplace=True)

In [67]:
display(attrition.isnull().any())

Id                         False
Age                        False
Attrition                  False
BusinessTravel             False
Department                 False
DistanceFromHome           False
Education                  False
EducationField             False
EnvironmentSatisfaction    False
Gender                     False
JobInvolvement             False
JobRole                    False
JobSatisfaction            False
MaritalStatus              False
MonthlyIncome              False
NumCompaniesWorked         False
OverTime                   False
PercentSalaryHike          False
PerformanceRating          False
StockOptionLevel           False
TotalWorkingYears          False
TrainingTimesLastYear      False
YearsAtCompany             False
YearsInCurrentRole         False
YearsSinceLastPromotion    False
YearsWithCurrManager       False
CommunicationSkill         False
Behaviour                  False
dtype: bool

In [68]:
attrition.corr()

,Id,Age,Attrition,DistanceFromHome,Education,EnvironmentSatisfaction,JobInvolvement,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,CommunicationSkill,Behaviour
Id,1.000000,-0.156452,0.694838,0.072305,-0.035610,-0.100908,-0.127778,-0.105379,-0.178736,0.008325,...,0.022698,-0.118617,-0.193855,-0.080326,-0.156880,-0.183577,-0.039062,-0.164596,0.083742,NaN
Age,-0.156452,1.000000,-0.200596,0.020658,0.219237,0.001950,0.073185,0.037604,0.509210,0.312099,...,0.002982,0.126257,0.675578,0.004789,0.355438,0.266728,0.222113,0.263792,0.019341,NaN
Attrition,0.694838,-0.200596,1.000000,0.109224,-0.041390,-0.111746,-0.159710,-0.148012,-0.240479,0.034763,...,0.010202,-0.175496,-0.262922,-0.105663,-0.230061,-0.269408,-0.050379,-0.222752,0.103379,NaN
DistanceFromHome,0.072305,0.020658,0.109224,1.000000,0.023945,-0.046390,0.016316,-0.097558,0.012521,-0.048131,...,0.071457,0.029378,0.029289,-0.032038,-0.030392,-0.010607,0.004460,-0.004135,-0.036109,NaN
Education,-0.035610,0.219237,-0.041390,0.023945,1.000000,-0.065041,0.083673,-0.036143,0.119907,0.091690,...,-0.036204,0.042585,0.175662,-0.043272,0.145911,0.076579,0.076430,0.121981,0.000223,NaN
EnvironmentSatisfaction,-0.100908,0.001950,-0.111746,-0.046390,-0.065041,1.000000,-0.000977,0.035033,-0.033990,-0.051681,...,-0.041427,-0.006479,-0.019005,0.020531,0.000685,0.008028,0.014325,0.002457,-0.011557,NaN
JobInvolvement,-0.127778,0.073185,-0.159710,0.016316,0.083673,-0.000977,1.000000,-0.005453,0.005859,0.012240,...,-0.063512,0.061673,0.037145,-0.027122,-0.008208,0.029460,0.006373,0.062552,0.052877,NaN
JobSatisfaction,-0.105379,0.037604,-0.148012,-0.097558,-0.036143,0.035033,-0.005453,1.000000,0.003255,-0.032738,...,-0.078554,0.024867,0.012892,-0.012757,0.023714,0.036065,0.004439,-0.006131,-0.051464,NaN
MonthlyIncome,-0.178736,0.509210,-0.240479,0.012521,0.119907,-0.033990,0.005859,0.003255,1.000000,0.169025,...,-0.037533,0.012281,0.759658,-0.020173,0.536715,0.418501,0.340080,0.401007,-0.060311,NaN
NumCompaniesWorked,0.008325,0.312099,0.034763,-0.048131,0.091690,-0.051681,0.012240,-0.032738,0.169025,1.000000,...,-0.015765,0.069008,0.226706,-0.043610,-0.099854,-0.081803,-0.046999,-0.116915,0.013289,NaN


In [69]:
categorical = []
for col,value in attrition.iteritems():
    if value.dtype == 'object':
        categorical.append(col)

numerical = attrition.columns.difference(categorical)
        

In [34]:
categorical


['BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'OverTime']

In [35]:
attrition_cat = attrition[categorical]
attrition_cat = pd.get_dummies(attrition_cat)
attrition_cat.head(3)

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,...,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,OverTime_No,OverTime_Yes
0,1,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
1,0,0,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,1
2,0,0,1,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,1,1,0


In [36]:
attrition_num = attrition[numerical]

In [45]:
attrition_final = pd.concat([attrition_num, attrition_cat], axis=1)
attrition_final.drop(['Id','Attrition'],axis=1,inplace=True)

In [46]:
from sklearn.model_selection import train_test_split
target=attrition['Attrition']
train, test, target_train, target_val = train_test_split(attrition_final,target,train_size= 0.80,random_state=0);

In [47]:
seed = 0   # We set our random seed to zero for reproducibility
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
#     'warm_start': True, 
    'max_features': 0.3,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}


In [48]:
rf = RandomForestClassifier(**rf_params)

In [49]:
rf.fit(train,target_train)
print("Fitting of Random Forest finished")

Fitting of Random Forest finished


In [50]:
rf_predictions = rf.predict(test)
print("Predictions finished")

Predictions finished


In [51]:
print("Accuracy score: {}".format(accuracy_score(target_val, rf_predictions)))
print("="*80)
print(classification_report(target_val, rf_predictions))

Accuracy score: 0.7883435582822086
              precision    recall  f1-score   support

           0       0.76      0.84      0.80       161
           1       0.82      0.74      0.78       165

    accuracy                           0.79       326
   macro avg       0.79      0.79      0.79       326
weighted avg       0.79      0.79      0.79       326



In [52]:
# Scatter plot 
trace = go.Scatter(
    y = rf.feature_importances_,
    x = attrition_final.columns.values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = attrition_final.columns.values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [62]:
# Gradient Boosting Parameters
gb_params ={
    'n_estimators': 1500,
    'max_features': 0.9,
    'learning_rate' : 0.25,
    'max_depth': 4,
    'min_samples_leaf': 2,
    'subsample': 1,
    'max_features' : 'sqrt',
    'random_state' : seed,
    'verbose': 0
}
gb = GradientBoostingClassifier(**gb_params)
# Fit the model to our SMOTEd train and target
gb.fit(train, target_train)
# Get our predictions
gb_predictions = gb.predict(test)
print("Predictions have finished")

Predictions have finished


In [63]:
print(accuracy_score(target_val, gb_predictions))
print(classification_report(target_val, gb_predictions))


0.9693251533742331
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       161
           1       0.94      1.00      0.97       165

    accuracy                           0.97       326
   macro avg       0.97      0.97      0.97       326
weighted avg       0.97      0.97      0.97       326



In [64]:
gb_predictions

array([1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [65]:
solution = pd.DataFrame({"id":test.Id, "Attrition":gb_predictions})
solution.to_csv("attrition.csv", index = False)

AttributeError: 'DataFrame' object has no attribute 'Id'